# Higher order functions and partial application

Our broader goal so far has been to achieve higher and higher levels of abstraction in our code. When programming, we study a domain of application, and translate its ideas into computational concepts (data and functions). In order for the translation to reach maximum usefulness, we need this translation in code to be as close as possible to the original ideas: code must be a _precise_ and _succint_ representation. Precision is important to avoid mistakes, and brevity is needed to avoid losing our train of thought in needless details which are not really present in the original domain, but are spuriously introduced by computational mechanisms. This way, we get to express *as much concepts with as little code as possible*, reducing both the effort it takes us to write our programs and the risk of making mistakes.

In order to achieve this domain-specificity of our code, we will need the ability to encode abstract concepts which do not directly map to concrete operations, but which make the expression of concrete operations much more logical for us to understand and reason about.

## Repetita iuvant
Let us start by observing two functions which are apparently unrelated in what they achieve, but which are based on the same underlying idea: adding numbers and writing lines.
```
sum := n =>
  s := 0
  while n > 0:
    s := s + n
    n := n - 1
  return s

line := n =>
  s := ""
  while n > 0:
    s := s + "*"
    n := n - 1
  return s
```

Even though the two functions clearly perform very different tasks (one produces a number, the other a line of asterisks), we can observe right away that they both share some structure. Specifically, both functions initialise variable `s` to some initial value (we consider this to be related to the base case of recursive functions), and then they both add a series of values which might depend on the current value of `n`. We can rewrite both functions by abstracting away the differing bits, so that the functions become identical, minus of course different definitions of the base step and the value to add at each iteration:

```
sum := n =>
  s := BASE
  while n > 0:
    s := s + STEP(n)
    n := n - 1
  return s

line := n =>
  s := BASE
  while n > 0:
    s := s + STEP(n)
    n := n - 1
  return s
```

The ability to identify similar structures gives an extra possibility: that of building a function that simply encapsulates this similar structure once and for all, so that instead of repeating it every time we can just invoke it ("refer to it"). After all, if a concept can be described, implemented, and tested once and for all, there is no point in rediscovering it instead of reusing it every time we need it!

We could call this shared structure `repeat_add`, since it embodies the idea of repeatedly adding values in a loop. This structure (which is no function yet), would look like:

```
repeat_add := n =>
  s := BASE
  while n > 0:
    s := s + STEP(n)
    n := n - 1
  return s
```

In order to turn this formulation into a function we would only need to somehow provide `BASE` and `STEP`. Base is clearly a value, so we might simply add it as a parameter. The other argument though is not so simple to handle: `STEP` depends on `n`, and as such we might recognize that it is a function. Fortunately for us, lambda expressions are no more than values, and as such they can be passed around just like normal parameters. This means that we can simply define `repeat_add` as a function as follows:

```
repeat_add := (n,BASE,STEP) =>
  s := BASE
  while n > 0:
    s := s + STEP(n)
    n := n - 1
  return s
```

Functions such as `repeat_add` use an important design pattern: they accept other functions as arguments (in this case `STEP`). Functions that accept other functions as arguments are called _higher order functions_, shortened as HOF. We can now instantiate `repeat_add` in order to obtain one-liner definitions of both `sum` and `line`:

```
sum := n => return repeat_add(n,0,x => return x)
line := n => return repeat_add(n,"",x => return "*")
```

`sum` will simply add each value from `n` down to `1`, starting from `0`. `line` is almost the same: it adds `n` asterisks, starting from the empty string `""`. Reading the definition of both functions now quite clearly states what the functions do, without having to read the much longer code that characterised their first implementation we gave in the beginning.

Moreover, since both `sum` and `line` (in accordance to the original definition, which we want to respect to the letter!) must take only `n` as a parameter, they both accept `n` as input and pass it through to `repeat_add`. Even though it makes sense, this bit of code that does nothing but pass a parameter through looks somewhat inelegant, and indeed it hampers the understandability of the functions. We will work on fixing this later.

Still, it is worthy of notice how, thanks to HOF's, we have achieved something significant: we have built our first function, `repeat_add`, that embodies an abstract concept such as repetition, without necessarily being bound to a concrete use-case. Our toolbox for expressing thought is growing, and when we do need to apply this toolbox to concrete problems (in our case `sum` and `add`), then it is immediately clear how powerful such a toolbox really is.

### Repeating actions
Armed with an extended arsenal for expressing abstract concepts, let us consider another function which is similar to the ones we have just built, yet sufficiently different to force us to take another upward step along the ladder of abstraction: 

```
mul := n =>
  s := 1
  while n > 0:
    s := s * n
    n := n - 1
  return s
```

Unfortunately this function cannot be implemented in terms of `repeat_add`, because instead of _adding_ the various values together, `mul` needs to multiply them.

In order to cover `mul` as well, we might generalize `repeat_add` even more, so that it simply accepts as input the operation it needs to perform between steps. We will call this operation `MERGE`, since it merges the current value of `s`, accumulated from the previous iterations, with the new value coming from `STEP(n)` which refers to the current iteration. We can see that both `repeat_add` and `mul` both adhere to this new formulation (and since `sum` and `line` are based on `repeat_add`, they can stay untouched):

```
repeat_add := n =>
  s := BASE
  while n > 0:
    s := MERGE(s, STEP(n))
    n := n - 1
  return s

mul := n =>
  s := BASE
  while n > 0:
    s := MERGE(s, STEP(n))
    n := n - 1
  return s
```

With this more powerful formulation we can create indeed an even more general `repeat` function that merges `s` and `STEP(n)`. We could also notice that instead of performing the `STEP` and its `MERGE` separately, we can perform them in one go, leading us to:

```
repeat := (n,BASE,MERGE_STEP) =>
  s := BASE
  while n > 0:
    s := MERGE_STEP(s, n)
    n := n - 1
  return s
```

We can now redefine `repeat_add` and `mul` by simply providing the proper parameters as follows:

```
repeat_add := (n,BASE,STEP) => return repeat(n, 0, (s,n) => return s + STEP(n))
mul := n => return repeat(n, 1, (s,n) => return s * n)
```

The `sum` and `line` functions could stay untouched (as they internally refer to `repeat_add`) but we can also redefine them in terms of `repeat`:

```
sum := n => return repeat(n, 0, (s,n) => return s + n)
line := n => return repeat(n, "", (s,n) => return s + "*")
```

We still have to deal with a minor annoyance though: the way we manage parameter `n`. It is clear that `n` takes on a different role than `BASE`, `STEP`, or `MERGE_STEP`: it remains a parameter *even after those others have been specified*. In order to express this layering of parameters, we can turn `repeat` into a function that further returns another function with `n` as parameter. This is made possible by the fact that functions are simply values, and as such they not only can be passed as parameters, but also returned:

```
repeat := (BASE,MERGE_STEP) =>
  return n =>
    s := BASE
    while n > 0:
      s = MERGE_STEP(s, n)
      n = n - 1
    return s
```

Now we can specify `BASE` and `MERGE_STEP` as the first parameters, without being forced to provide a fake `n` which we just pass-through:

```
repeat_add := (BASE,STEP) => return repeat(BASE, (s,n) => return s + STEP(n))
sum  := repeat(0, (s,n) => return s + n)
line := repeat("", (s,n) => return s + "*")
mul  := repeat(1, (s,n) => return s * n)
```

Of course this translates quite directly to Python, minus some minor syntax adjustments:

In [2]:
def repeat(base,merge_step):
    def f(n):
        s = base
        while n > 0:
            s = merge_step(s,n)
            n = n - 1
        return s
    return f
repeat_add = lambda base,step: repeat(base,lambda s,n:s + step(n))

sum  = repeat_add(0,  lambda n: n)
line = repeat_add("", lambda n: "*")
sumR  = repeat(0,   lambda s,n: s + n)
lineR = repeat("",  lambda s,n: s + "*")
mul   = repeat(1,   lambda s,n: s * n)

n = int(input())
print(sum(n))
print(line(n))
print(sumR(n))
print(lineR(n))
print(mul(n))

4
10
****
10
****
24


## Composing functions

Let us reformulate our original goal: we look for ways of expressing concepts from the real world (or better yet: the internal model that we as human beings have of the real world) into concepts familiar to a programming language. Some real-world concepts are very concrete, such as _Jim's age_ or _drawing a line_. Some real-world concepts, such as _repeating an action n times_, are less concrete, because they do not correspond to a _thing_ or _action_ themselves, but exist on a higher level of abstraction. We could call _repeat_ a _meta-action_ (or HOF).

As our last example of this phenomenon, let us consider the most basic way of combining actions (functions) together. In the real-world we can define the concept of _A then B_, where the meta-action `then` performs action _B_ right after action _A_. The sequentiality implies that whatever _A_ did is made available to _B_ in order for _B_ to perform its own action.

In order to turn this real-world idea into code, we could observe that `then` takes as input two functions, which are then invoked in sequence in order to create a pipeline.

The implementation seems almost trivial:

```
then := (f,g) => 
  return (x => return g(f(x)))
```

Let us see this in action. Consider simple functions such as:

```
incr := x => return x + 1
double := x => return x * 2
```

Suppose now that we wanted to increment a given number twice. Thanks to `then`, we can simply say:

```
incr_twice := then(incr,incr)
```

We could also increment three times:

```
incr_thrice := then(incr,then(incr,incr))
```

or even:

```
incr_thrice := then(incr,incr_twice)
```

We are clearly not bound to only using the same function multiple times. For example, we could define a function to first increment, and then double a value as:

```
incr_and_double: then(incr, double)
```

And so on:

```
then(incr, then(double, incr))
```

The power of the `then` operator is unfortunately too easy to understate. This apparently unassuming operator is the key to building (potentially very long) chains or pipelines of functions which simply take over from the previous, perform their action, and then delegate the rest of the work to the next steps in the pipeline. 

This ability to transform data from input to output captures the essence of programming, and thus an operator which is central to clearly defining such transformation pipelines is also very close to the heart of programming.

In conclusion, thanks to HOF's, our toolbox is not only growing, but our ability to express and compose abstract concepts is growing as well. Composition is of quintessential importance: without composition, our ability to connect concepts together is limited, and so is our ability to represent _networks of ideas_, which characterize most interesting and articulated problem domains.